In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, load_model
import numpy as np
import tensorflow as tf
from glob import glob

In [2]:
tf.debugging.set_log_device_placement(True)

In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices(device_type='GPU'))

2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
dataset_path = './dataset'

In [10]:
train_dataset_path = dataset_path + '/train'
valid_dataset_path = dataset_path + '/test'

In [11]:
IMAGE_SIZE = [224, 224]

In [12]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in

In [13]:
for layer in inception.layers:
  layer.trainable = False

In [14]:
folders = glob(train_dataset_path + '/*')
print(folders)

['./dataset/train\\def_front', './dataset/train\\ok_front']


In [15]:
x = Flatten()(inception.output)

prediction = Dense(len(folders), activation='sigmoid')(x)

model = Model(inputs=inception.input, outputs=prediction)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device

In [16]:
x

<KerasTensor: shape=(None, 51200) dtype=float32 (created by layer 'flatten')>

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [18]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [19]:
train_dataset = train_data_gen.flow_from_directory(train_dataset_path,
                                                   target_size=(224, 224),
                                                   batch_size=64,
                                                   class_mode='categorical')

test_dataset = test_data_gen.flow_from_directory(valid_dataset_path,
                                                   target_size=(224, 224),
                                                   batch_size=64,
                                                   class_mode='categorical')

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


In [20]:
r = model.fit(train_dataset,
              validation_data=test_dataset,
              epochs=5,
              steps_per_epoch=len(train_dataset),
              validation_steps=len(test_dataset))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp i

In [21]:
model.save('./models/my_model.h5')

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /j

In [22]:
from PIL import Image
import numpy as np
from skimage import transform
import os

In [23]:
def load_image(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (224, 224, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

In [24]:
def_dir = './dataset/test/def_front'
ok_dir = './dataset/test/ok_front'

In [25]:
model = load_model('./models/my_model.h5')

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in d

In [26]:
def_dir_list = os.listdir(def_dir)
def_dir_list

['cast_def_0_1059.jpeg',
 'cast_def_0_1063.jpeg',
 'cast_def_0_108.jpeg',
 'cast_def_0_1096.jpeg',
 'cast_def_0_112.jpeg',
 'cast_def_0_1134.jpeg',
 'cast_def_0_1136.jpeg',
 'cast_def_0_1137.jpeg',
 'cast_def_0_1153.jpeg',
 'cast_def_0_1171.jpeg',
 'cast_def_0_1172.jpeg',
 'cast_def_0_1174.jpeg',
 'cast_def_0_118.jpeg',
 'cast_def_0_1186.jpeg',
 'cast_def_0_1189.jpeg',
 'cast_def_0_1191.jpeg',
 'cast_def_0_1203.jpeg',
 'cast_def_0_1211.jpeg',
 'cast_def_0_1215.jpeg',
 'cast_def_0_1231.jpeg',
 'cast_def_0_1233.jpeg',
 'cast_def_0_1238.jpeg',
 'cast_def_0_1239.jpeg',
 'cast_def_0_1258.jpeg',
 'cast_def_0_1260.jpeg',
 'cast_def_0_1264.jpeg',
 'cast_def_0_1269.jpeg',
 'cast_def_0_1291.jpeg',
 'cast_def_0_1292.jpeg',
 'cast_def_0_1294.jpeg',
 'cast_def_0_1298.jpeg',
 'cast_def_0_1320.jpeg',
 'cast_def_0_1323.jpeg',
 'cast_def_0_1324.jpeg',
 'cast_def_0_1351.jpeg',
 'cast_def_0_1352.jpeg',
 'cast_def_0_1358.jpeg',
 'cast_def_0_1359.jpeg',
 'cast_def_0_1376.jpeg',
 'cast_def_0_1379.jpeg',
 'c

In [27]:
result = []
for file in def_dir_list:
    image = load_image(def_dir + '/' + file)
    pred = model.predict(image)
    result.append(np.argmax(pred))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptionsDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Opt

In [28]:
print(f'True Defect : {result.count(0)} | False Defect : {result.count(1)} | Accuracy : {result.count(0)/(result.count(0) + result.count(1)) * 100}%')

True Defect : 451 | False Defect : 2 | Accuracy : 99.55849889624724%


In [ ]:
ok_dir_list = os.listdir(ok_dir)
ok_dir_list

In [ ]:
result = []
for file in ok_dir_list:
    image = load_image(ok_dir + '/' + file)
    pred = model.predict(image)
    result.append(np.argmax(pred))

In [ ]:
print(f'True OK : {result.count(1)} | False OK : {result.count(0)} | Accuracy : {result.count(1)/(result.count(0) + result.count(1)) * 100}%')